# Satellite and Formation Image Collection

Within this program, given a list of area names and latitude and longitude pairs, will construct images that are used as the source for the image collection for the training, testing and validation images for the GAN that is used in this project. The program will also output the latitude and longitude values for the top-left point of the images constructed.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import contextily as cx
%matplotlib inline

import math

import os
import urllib.request
from PIL import Image

## Input Data

The following two cells contains all the variables that you have the option to change.

In [2]:
# Path to where you want to save off your images
save_path = "Collected_Images"

# How you want to name the resulting folder
folder_name = "DM_Dataset"

# Mapbox access_token
access_token = "pk.eyJ1IjoiY2xhd3NvbjEwMSIsImEiOiJja2Iwdzg3cHMwMHZvMnZvODBncDRjbjBiIn0.PeYOCiVXGZ_7oJLvKFL28Q" # FILL IN WITH YOUR OWN MAPBOX ACCESS TOKEN
#Remove when publishing

# Zoom of the images
z = 9

# How much you want to crop from the top, left, right and bottom of
# the resulting images
crop = 0

# WORD THIS BETTER
size = 3

In [3]:
# The tuples consisting of the 1) Name, 2) Latitude, and 3) Longitude
# data to be used
data = [
    (        "Chihuahua_N", 29.7429, -106.1115),    # Desert
    (              "Texas", 31.2386, -106.0703),    # Desert
    (           "SE_Idaho", 42.7551, -112.3792),    # Mountain
    (         "W_Virginia", 37.2522,  -81.3620),    # Mountain
    ("New_Mexico_Franklin", 33.0691, -106.5550),    # Mountain
    (           "Colorodo", 37.9128, -107.1098),    # Mountain
    (            "W_Texas", 31.2306, -103.2689),    # Desert
    (     "Mid_New_Mexico", 35.0975, -104.6570),    # Desert
    (       "W_New_Mexico", 33.3027, -105.9667),    # Mountain
    (       "S_New_Mexico", 32.6556, -110.7504),    # Desert
    (          "S_Arizona", 33.2574, -113.9307),    # Desert
    (       "S_California", 34.4973, -115.0333),    # Desert
    (         "Mid_Nevada", 39.5407, -116.5303),    # Desert
    (         "N_Colorodo", 40.3758, -107.3886),    # Mountain
    (         "S_Virginia", 38.2043,  -79.4638),    # Mountain
    (             "N_Utah", 40.3507, -110.8163),    # Mountain 
    (           "N_Nevada", 40.9031, -115.8728),    # Desert
    (          "NW_Nevada", 40.8512, -118.5919),    # Desert
    (          "Mid_Idaho", 45.2169, -113.8129),    # Mountain
    (            "N_Idaho", 47.9623, -113.4613),    # Mountain
    (       "Pennsylvania", 40.8221,  -78.3875),    # Mountain
    (         "Mid_Mexico", 27.0224, -104.0845),    # Desert
    (          "Monterrey", 25.3192, -100.6952),    # Desert
    (             "Sonora", 29.3199, -110.3549),    # Mountain
    (        "W_Chihuahua", 29.4994, -107.7621),    # Mountain
    (           "Coahuila", 28.3406, -102.2717),    # Mountain
    (            "Durango", 25.2671, -103.7082),    # Desert
    (         "Guanajuato", 20.8742, -101.3654),    # Desert
    (          "Zacatecas", 23.5464, -103.2907),    # Desert
    (            "Hidalgo", 20.7458,  -98.7341),    # Mountain
    (            "Morelos", 18.6879,  -99.2532),    # Desert
    (             "Oaxaca", 17.1959,  -96.7072),    # Mountain
    (           "W_Canada", 50.4120, -114.7906),    # Mountain
    (          "NW_Canada", 53.2718, -118.9902),    # Mountain
    (           "S_Mexico", 19.3163, -101.5027),    # Desert
    (          "N_Arizona", 35.9736, -111.3657),    # Desert
    (       "MidW_Arizona", 34.6129, -113.1070),    # Desert
    (           "N_Oregon", 44.5474, -120.9073),    # Desert
    (      "SW_Washington", 47.1505, -118.6688),    # Desert
    (        "Mid_Montana", 46.9803, -109.9127),    # Mountain
    (         "SE_Montana", 46.2083, -107.8308),    # Desert
    (     "SE_Mid_Montana", 46.4189, -105.8203),    # Desert
    (            "N_Texas", 33.5391,  -99.8932),    # Desert
    (        "S_Mid_Idaho", 44.2275, -114.6643),    # Mountain
    (          "N_Montana", 48.0046, -110.0555),    # Desert
    (          "Mid_Texas", 30.7749,  -99.7449),    # Desert
    (            "S_Texas", 28.3962,  -99.0115),    # Desert
    (       "W_Tamaulipas", 23.9310, -100.1926),    # Mountain
    (           "N_Sinola", 26.6327, -107.6468),    # Mountain
    (        "W_Zacatecas", 23.9762, -104.1064),    # Mountain
    (         "MidW_Texas", 30.9352, -101.9009),    # Desert
    (        "Mid_Arizona", 34.6038, -110.9317),    # Desert
    (          "S_Wyoming", 42.3342, -109.2068),    # Desert
    (         "NMid_Idaho", 44.8831, -114.4775),    # Mountain
    (             "S_Iran", 30.0786,   56.5439),    # Desert
    (            "SW_Iran", 29.7858,   59.3042),    # Mountain
    (           "Mid_Iran", 32.1245,   58.8181),    # Desert
    (             "N_Iran", 33.5849,   59.1064),    # Desert
    (               "Iran", 32.4889,   56.6235),    # Desert
    (         "Afganistan", 33.6672,   66.0416),    # Mountain
    (            "NW_Iran", 33.0524,   48.6859),    # Mountain
    (          "NMid_Iran", 32.3846,   54.4427),    # Desert
    (            "NE_Iran", 36.4058,   58.8950),    # Mountain
    (            "SE_Iran", 29.0874,   57.5079),    # Mountain
    (      "NE_Afganistan", 35.8779,   69.4830),    # Mountain
]

## Functionality of image collection

The sources that we will be pulling satellite and formation images from is Mapbox and Macrostrat respectively. The way that both Mapbox and Macrostrat store and organize their images is through "tiles." Each tile can be accessed with a latitude and longitude pair with a desired zoom. 

The following two functions are related to accessing the correct tiles: 

 - The first function accesses the correct tile given a latitude-longitude pair. 

- The second function deals with finding what the latitude-longitude pair is for the top left corner of the given tile.

In [4]:
'''
    Will return the x and y tile for a given latitude and longitude pair
    along with the zoom.
    
    :param lat_deg: float, latitude in degrees
    :param lon_deg: float, longitude in degrees
    :param zoom: integer, zoom for image
    :return: zoom of tile, xtile, ytile
'''

def deg2num(lat_deg, lon_deg, zoom):
    lat_rad = math.radians(lat_deg)
    n = 2.0 ** zoom
    xtile = int((lon_deg + 180.0) / 360.0 * n)
    ytile = int((1.0 - math.asinh(math.tan(lat_rad)) / math.pi) / 2.0 * n)
    return (zoom, xtile, ytile)

In [5]:
'''
    This function will output the latitude and longitude data for top-left
    point for the given latitude-longitude pair given.
    
    :param lat: Starting Latitude value
    :param lng: Starting Longitude value
    :param z: Desired zoom of the tile
    :t_x: The desired x tile
    :t_y: The desired y tile
    :return: Zoom, Latitude coordinate of the top-left, Longitude coordinate of the top-left
'''

def nearest_to(lat, lng, z, t_x, t_y):
    
    # First the program aquires the current x and y tiles
    # and compares it to the desired x and y tiles
    z, cur_x, cur_y = deg2num(lat, lng, z)
    temp_lat = lat
    temp_lng = lng
    in_lat = cur_x == t_x
    in_lng = cur_y == t_y
    
    # If either of the tiles are different, then increment/decrement the
    # starting latitude or longitude (depending on the case) until
    # the current latitude and current longitude coordinates are
    # on the desired x and y tiles
    while(not (in_lat and in_lng)):
        if(cur_x < t_x):
            temp_lng += 0.0001
        elif(cur_x > t_x):
            temp_lng -= 0.0001
            
        if(cur_y < t_y):
            temp_lat += 0.0001
        elif(cur_y > t_y):
            temp_lat -= 0.0001
            
        z, cur_x, cur_y = deg2num(lat, lng, z)
        in_lng = cur_x == t_x
        in_lat = cur_y == t_y
    
    
    # Next we want to get the latitude-longitude pair for the top
    # left of the current tile.
    
    # The way this will work is we have two "trip" variables that will
    # trip once the current latitude or longitude coordinates map to a different
    # x or y tile.
    x_trip = False
    y_trip = False
    
    # Lastly, because to get to the top left, you need to both decrease the
    # longitude and increase the latitude, it's just a matter of incrementing/
    # decrementing the respective coordinates until both x and y tiles are tripped
    while(not(x_trip and y_trip)):
        
        if(not(x_trip)):
            temp_lng -= 0.0001
        if(not(y_trip)):    
            temp_lat += 0.0001
        
        z, cur_x, cur_y = deg2num(temp_lat, temp_lng, z)
        x_trip = not(cur_x == t_x)
        y_trip = not(cur_y == t_y)
        
    # Then because we didn't update them after they tripped, increase them to
    # be on the correct tile
    temp_lat -= .0001
    temp_lng += .0001
    
    return (z, temp_lat, temp_lng)
    
    

## Setting up

Here we create the necessary local variables and directories to make sure that everything saves off correctly.

In [6]:
# Setting up the necessary directories in order to save off images correctly

os.makedirs("%s" % (save_path), exist_ok=True)
os.makedirs("%s/%s" % (save_path, folder_name), exist_ok=True)

# Will contain the pure formation images
os.makedirs("%s/%s/Form_pics" % (save_path, folder_name), exist_ok=True)

# Will conatin the pure satellite images
os.makedirs("%s/%s/SAT_pics" % (save_path, folder_name), exist_ok=True)

# Will contain the GAN source images
os.makedirs("%s/%s/GAN_pics" % (save_path, folder_name), exist_ok=True)

# Setting up save paths that were established in the directories
save_paths = [
                '%s/%s/SAT_pics/' % (save_path, folder_name),
                '%s/%s/Form_pics/' % (save_path, folder_name),
                '%s/%s/GAN_pics/' % (save_path, folder_name)
             ]

# Constructing the mapbox api request to access the satelite tiles
first = "https://api.mapbox.com/styles/v1/mapbox/satellite-v9/tiles/"
second = "%d/%d/%d?access_token="

url = first+second+access_token

# Organizing the url requests that are going to be called for each
# area. The first url is the satelite image request, the second is the
# macrostrat request for the formation images.

urls = [
            url,
            "https://tiles.macrostrat.org/carto/%d/%d/%d.png"
       ]


# These two arrays will contain the latitude and longitude coordinates
# of the top left corner of the given tile
out_lat_data = []
out_lng_data = []

## Collecting and Saving the GAN Source Images

First loop will go through the data in order to get the coresponding x and y tile, then the program will get the surrounding 8 tiles to the initial tile then it will be saved off. This process is repeated for the satellite images and the formation images. The resulting images is a 3x3 tile image with the requested latitude and longitude data being inside the middle tile.

In [7]:
for i in range(0, len(data)):
    name = data[i][0]
    lat = data[i][1]
    lng = data[i][2]
    
    # Getting to correct x and y tile for the center tile of the resulting
    # Image.
    z, x, y = deg2num(lat, lng, z)
    
    # Getting the top left latitude and longitude coordinate for the
    # center tile and saving it
    z, lat, lng = nearest_to(lat, lng, z, x, y)

    out_lat_data.append(round(lat,4))
    out_lng_data.append(round(lng,4))
    
    # Setting up the names for both the satellite names and the formation
    # images.
    names = [
                name + "_sat",
                name + "_form"
            ]

    # Lastely, we go through the request urls, the mapbox request and the
    # macrostrat request.
    for k in range(0, len(urls)):
        
        # Here is where the 9 tiles will be saved off to then be stitched
        # together for the resulting image.
        imgs = []
        
        # Make the necessary request to the respective url and save it
        # to the images.
        for i in range (-1 * (size // 2), (size // 2) + 1, 1):
            for j in range(-1 * (size // 2), (size // 2) + 1, 1):
                imgs.append(Image.open(urllib.request.urlopen(urls[k] % (z,x+i,y+j))))

        
        # Next, because the images are going to be requested in a left-to-right
        # top-to-bottom fashion, the first image in imgs will be the top-left
        # image, the second will be the top-center image and etc.
        
        w1, h1 = imgs[0].size
        
        jnt_size = int(math.sqrt(len(imgs)))
        
        jnt = Image.new("RGB", (jnt_size * w1, jnt_size* h1))

        for i in range(0, len(imgs)):
            jnt.paste(imgs[i], (w1 * (i // size), h1 * (i % size)))

        # So stitch them all together in order and then save them off with
        # the approprate name and save path
        
        jnt.save(save_paths[k] + names[k] + ".png")


C:\Users\lawso\AppData\Local\Continuum\anaconda3\envs\geospatial\lib\site-packages\PIL\Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [8]:
# Lastly, for all the images in the data we want to construct the final
# GAN source image that will be used to construct the training, testing, and
# validation images.

for i in range(0, len(data)):
    name = data[i][0]
    
    # For this we will open the satellite image and formation image
    sat_img = Image.open(save_paths[0] + name + "_sat.png")
    form_img = Image.open(save_paths[1] + name + "_form.png")
    
    w, h = sat_img.size  
    
    # Crop off however much is desired
    l = 0 + crop
    t = 0 + crop
    r = w - crop
    b = h - crop
    
    sat_img = sat_img.crop((l, t, r, b))
    form_img = form_img.crop((l, t, r, b))
    
    w, h = sat_img.size
    
    # Then stitch the two images together and save them off
    joint = Image.new("RGB", (w * 2, h))
    joint.paste(sat_img, (0,0))
    joint.paste(form_img, (w,0))
    
    joint.save(save_paths[2] + name + "-gan.png")

## Saving off Latitude-Longitude Data

Here we save off the latitude-longitude data associated with the top-left corner of the requested latitude-longitude data to help with aligning images with other services.

In [9]:
names_data = []

for i in range(0, len(data)):
    names_data.append(data[i][0])

export_data = {
                "Area Name": names_data,
                "Latitude Data": out_lat_data,
                "Longitude Data": out_lng_data
              }

export = pd.DataFrame(data=export_data)
export.to_excel(save_path + "/Mapbox_Data.xlsx", index = False)